## Scraping MLS Player Stats

Scraping www.mlssoccer.com/ for player statistics

### Import Necessary Modules

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime 
from dateutil import parser
import os
import time
import itertools
import re
import pickle

### Scrape Statistics

#### Define Scraper and Helpers

In [3]:
def send_request(url, params={}, attempts=3):

    cnt = 0
    while cnt <= attempts:
        cnt += 1        
        # Send Request
        response = requests.get(url, params)
        # Check Status
        print(response.url, '\n', response.status_code, response.reason)
        if (response.status_code >= 200) and (response.status_code < 300):           
            return response
        # If bad status, pause before trying again
        print('Pause, then retry')
        time.sleep(15)
    return None

In [4]:
def map_id_to_club(franchise_id):
    
    franchise_map = {11091: 'ATL',
                     1207: 'CHI',
                     436: 'COL',
                     454: 'CLB',
                     1326: 'DC',
                     1903: 'DAL',
                     1897: 'HOU',
                     1230: 'LA',
                     11690: 'LAFC',
                     6977: 'MNUFC',
                     1616: 'MTL',
                     928: 'NE',
                     9668: 'NYCFC',
                     399: 'NYRB',
                     6900: 'ORL',
                     5513: 'PHI',
                     1581: 'POR',
                     1899: 'RSL',
                     1131: 'SJ',
                     3500: 'SEA',
                     421: 'KC',
                     2077: 'TOR',
                     1708: 'VAN'}
    return franchise_map[franchise_id]
        

In [5]:
def check_for_data(stats_table):
    # Check if the table has any 
    odd_rows = stats_table.findAll('tr', {'class': 'odd'})
    if odd_rows[0].text.strip() == 'Stats Unavailable':
        return False
    return True    

In [6]:
def split_player_name(df):
    first = []
    last = []
    for t in df.Player.str.split():
        if len(t) == 1:
            first.append(t[0])
            last.append('')
        elif len(t) == 2:
            first.append(t[0])
            last.append(t[1])
        else:
            first.append(t[0])
            last.append(' '.join(t[1:]))
    
    df['Last'] = last
    df['First'] = first  
    
    df = df.drop('Player', axis=1)
    
    return df
        

In [7]:
# # Find and parse franchise popup menu to get list of teams
# soup = BeautifulSoup(response.text, 'lxml')
# franchise_select = soup.find('select', {'id': 'edit-franchise', 'name': 'franchise'})
# franchise = []
# # start with second index since first is generic "Select A Club"
# for team in franchise_select.findAll('option')[1:]:
#     franchise.append(int(team['value']))

In [8]:
def generate_param_combos(param_groups):
    params = []
    for combo in itertools.product(*param_groups):
        params.append({'franchise': combo[0],
                       'group': combo[1],
                       'season_type': combo[2],
                       'year': combo[3],
                       'page': combo[4]})
    return params

In [9]:
def scrape_player_stats(base_url, params, return_last_pg=False):

    response = send_request(base_url, params)
    if response is None:
        return None
    
    # Parse HTML
    soup = BeautifulSoup(response.text,'lxml')
    
    stats_table = soup.find('table')
    
    if not check_for_data(stats_table):
        return None
    
    stats_df = extract_stats(stats_table)
    # Add Year Column
    stats_df['Year'] = np.repeat(int(params['year']), len(stats_df))
    # Add Season Column
    stats_df['Season'] = np.repeat(params['season_type'], len(stats_df))
    
    if return_last_pg:
        return stats_df, get_last_page(soup)
    
    return stats_df     

In [15]:
def extract_stats(stat_table):
    
    # Extract salary data
    stat_header = []
    stat_data = []
    for row in stat_table.findAll('tr'):
        row_data = []

        # Get row type and check if header or data row
        row_type = row.findChild().name
        if row_type == 'th':
            # Extract header
            for h in stat_table.findAll('th'):
                stat_header.append(h.text) 
        else:
            # Extract data
            for data in row.findAll('td'):
                row_data.append(data.text)
            stat_data.append(row_data)    
            
    # Compile stat dataframe
    stat_df = pd.DataFrame(stat_data, columns=stat_header)
    
    # Strip any whitespace from column names
    stat_df.columns = stat_df.columns.str.strip()
    
    # Replace blanks with nans
    stat_df = stat_df.replace('', np.nan)
        
    return stat_df

In [16]:
def get_last_page(soup):

    last_pg_url = soup.find('li', {'class': 'pager-last last'}).find('a')['href']
    last_pg = int(re.search('(?<=page=)\d+', last_pg_url).group())
    
    return last_pg

In [17]:
def scrape_group_stats(base_url, group, year=np.arange(2007, 2018), franchise='select', season_type='REG'):
    
    group_df = pd.DataFrame()
    combos = generate_param_combos([[franchise], [group], [season_type], year, [0]])
    for params in combos: 
        stats_df, last_pg = scrape_player_stats(base_url, params, True)
        
        if stats_df is None:
            continue
            
        # Scrape first page of results
        group_df = pd.concat([group_df, stats_df], axis=0)
        # Add pause to prevent 429 status
        print(datetime.now())
        time.sleep(np.random.uniform(2, 5)) 
        
        # Scrape next through last
        for idx in range(1, last_pg+1):
            params['page'] = idx
            stats_df = scrape_player_stats(base_url, params)
            if stats_df is None:
                continue
            group_df = pd.concat([group_df, stats_df], axis=0)            
            # Add pause to prevent 429 status
            print(datetime.now())
            time.sleep(np.random.uniform(2, 5)) 
            
    # Reset index to remove duplicates created during concatenation
    group_df.reset_index(inplace=True, drop=True)
    
    return group_df        

#### Send Request to Base URL and Verify Site is Up

In [18]:
base_url = 'http://www.mlssoccer.com/stats/season'
response = requests.get(base_url)
response.status_code, response.url

(200, 'https://www.mlssoccer.com/stats/season')

#### Define parameter options

In [19]:
# group = ['goals', 'assists', 'shots', 'fouls', 'goalkeeping']
# for grp in group:
#     print('Scraping:', grp, datetime.now(), '\n')
#     df = scrape_group_stats(base_url, grp, np.arange(2007, 2018, 1))
#     df.to_pickle(grp + '_df.pkl')
#     print('Completed Scraping:', grp)

In [20]:
fouls_df = scrape_group_stats(base_url, 'fouls')

https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=0 
 200 OK
2018-01-26 12:07:31.171136
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=1 
 200 OK
2018-01-26 12:07:36.497680
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=2 
 200 OK
2018-01-26 12:07:44.442345
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=3 
 200 OK
2018-01-26 12:07:50.823871
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=4 
 200 OK
2018-01-26 12:07:58.925104
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=5 
 200 OK
2018-01-26 12:08:04.272584
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2007&page=6 
 200 OK
2018-01-26 12:08:09.462353
https://www.mlssoccer.com/stats/season?fr

https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2010&page=13 
 200 OK
2018-01-26 12:13:51.245107
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2010&page=14 
 200 OK
2018-01-26 12:13:57.170077
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2010&page=15 
 200 OK
2018-01-26 12:14:03.912103
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2010&page=16 
 200 OK
2018-01-26 12:14:09.255795
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2011&page=0 
 200 OK
2018-01-26 12:14:15.702111
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2011&page=1 
 200 OK
2018-01-26 12:14:21.623083
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2011&page=2 
 200 OK
2018-01-26 12:14:27.198047
https://www.mlssoccer.com/stats/seaso

https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=14 
 200 OK
2018-01-26 12:20:55.860092
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=15 
 200 OK
2018-01-26 12:21:00.881560
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=16 
 200 OK
2018-01-26 12:21:08.197260
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=17 
 200 OK
2018-01-26 12:21:15.915097
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=18 
 200 OK
2018-01-26 12:21:22.210726
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=19 
 200 OK
2018-01-26 12:21:29.185081
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2013&page=20 
 200 OK
2018-01-26 12:21:35.682538
https://www.mlssoccer.com/stats/se

https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=8 
 200 OK
2018-01-26 12:27:39.642513
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=9 
 200 OK
2018-01-26 12:27:47.567199
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=10 
 200 OK
2018-01-26 12:27:54.114285
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=11 
 200 OK
2018-01-26 12:28:00.308901
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=12 
 200 OK
2018-01-26 12:28:05.771139
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=13 
 200 OK
2018-01-26 12:28:10.354593
https://www.mlssoccer.com/stats/season?franchise=select&group=fouls&season_type=REG&year=2016&page=14 
 200 OK
2018-01-26 12:28:17.489689
https://www.mlssoccer.com/stats/seas

In [21]:
fouls_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5041 entries, 0 to 5040
Data columns (total 17 columns):
Player    5041 non-null object
Club      5037 non-null object
POS       5041 non-null object
GP        5037 non-null object
GS        5037 non-null object
MINS      5037 non-null object
G         5037 non-null object
A         5037 non-null object
SHTS      5037 non-null object
SOG       5037 non-null object
FC        5037 non-null object
FS        5037 non-null object
OFF       5037 non-null object
YC        5037 non-null object
RC        5037 non-null object
Year      5041 non-null int64
Season    5041 non-null object
dtypes: int64(1), object(16)
memory usage: 669.6+ KB


In [22]:
fouls_df

,Player,Club,POS,GP,GS,MINS,G,A,SHTS,SOG,FC,FS,OFF,YC,RC,Year,Season
0,Bobby Boswell,DC,D,23,22,1963,0,2,7,2,42,4,0,5,1,2007,REG
1,Facundo Erpen,COL,D,26,25,2056,1,2,11,2,42,23,0,6,1,2007,REG
2,Khano Smith,NE,M,29,26,2210,2,6,36,12,42,41,5,4,0,2007,REG
3,Bakary Soumare,CHI,D,19,11,1072,0,0,4,1,41,17,0,6,1,2007,REG
4,Adam Cristman,NE,F,28,14,1421,4,4,40,16,41,51,10,1,0,2007,REG
5,Carlos Marinelli,KC,M,26,19,1674,1,5,42,21,41,41,1,7,0,2007,REG
6,Mehdi Ballouchy,COL,M,23,22,1853,1,2,35,10,40,39,5,8,1,2007,REG
7,Ugo Ihemelu,COL,D,25,23,2017,1,0,7,4,40,34,0,4,2,2007,REG
8,Brian Carroll,DC,M,28,19,1801,0,4,7,1,39,15,1,7,0,2007,REG
9,Kyle Beckerman,RSL,M,28,27,2430,3,3,52,19,39,41,2,9,0,2007,REG


In [23]:
fouls_df.to_pickle('fouls_df.pkl')

### Sandbox